# Notebook Audit Database Mesin dan Cleaning Database Mesin

Import Library

In [18]:
import pandas as pd
import numpy as np

Load Data

In [19]:
# Load Dataframe
print("⏳ Sedang membaca file Excel...")
df_master = pd.read_excel('1_Master_Aset_Aktif.xlsx')
df_history = pd.read_excel('2_Riwayat_Log_Fix.xlsx')


print(f"Jumlah Aset Aktif: {len(df_master)} baris")
print(f"Jumlah History: {len(df_history)} baris")

⏳ Sedang membaca file Excel...
Jumlah Aset Aktif: 6100 baris
Jumlah History: 10821 baris


Cek Variasi Kategori

In [20]:
# Gabungkan kolom kategori dari kedua file
cat_master = df_master['kategori']
cat_history = df_history['kategori']
all_kategori = pd.concat([cat_master, cat_history]).dropna()

# Ubah ke uppercase dan strip spasi biar bersih
all_kategori = all_kategori.astype(str).str.upper().str.strip()

# Hitung kemunculan dan urutkan abjad
print("--- 🔍 VARIASI KATEGORI (URUT ABJAD) ---")
# Menampilkan semua variasi unik beserta jumlahnya
# Format: Nama Kategori (Jumlah Data)
variasi_kategori = all_kategori.value_counts().sort_index()

# Tampilkan di output cell (karena ini ipynb, dia akan muncul sbg tabel)
pd.set_option('display.max_rows', None) # Agar semua baris terlihat
display(variasi_kategori)

--- 🔍 VARIASI KATEGORI (URUT ABJAD) ---


kategori
((  KURSI PIJAT ))                                                        17
((  PHOTOBOOTH ))                                                          1
(( FIXED GAME ))                                                         572
(( FIXED GAME / FIXED GAME ))                                             10
(( FIXED GAME / KIDDIE RIDE ))                                            62
(( FIXED GAME))                                                           10
(( FIXED GAME/KIDDIE RIDE))                                               15
(( KARAOKE )                                                              12
(( KARAOKE ))                                                            121
(( KARAOKE - BARANG SAJA/UNIT CPU ))                                      20
(( KARAOKE - REAL TIME/KESATUAN))                                          4
(( KARAOKE ROOM ))                                                         2
(( KIDDIE LAND ))                                                  

Cek Variasi Lokasi Toko

In [21]:
# Gabungkan kolom lokasi
loc_master = df_master['lokasi_toko']
loc_history = df_history['lokasi_asal'] # Ingat, nama kolomnya beda di file history
all_lokasi = pd.concat([loc_master, loc_history]).dropna()

# Bersihkan
all_lokasi = all_lokasi.astype(str).str.upper().str.strip()
print("---  VARIASI LOKASI  ---")
variasi_lokasi = all_lokasi.value_counts().sort_index()

pd.set_option('display.max_rows', None) 
display(variasi_lokasi)

---  VARIASI LOKASI  ---


R 100 CRBN                   210
R 102 KBL2                   215
R 105 - PRG                  424
R 121 - TAJUR                135
R 124 - CIKUPA               112
R 125 - JTNG                 215
R 135 - SS TANGERANG          70
R 15 BGR2                    149
R 20 CPUTAT                  217
R 21 BKSI                    304
R 22 PSMGU                   178
R 29 - TJPR                  383
R 31 - TANGGERANG            373
R 34 DPK                     465
R 35 CLGN                    504
R 37 CBTG                    407
R 38 KLD                     897
R 49 CLDG                    465
R 67 KARWG                   556
R 71 BTM                     231
R 75 SRNG                    383
R106_-_CBDK                  201
R114 - BGR JASMIN            120
R138_-_TIMIKA                 83
R26 - CIMONE                 127
R40 CKRG                     410
RB 31 BG JUCTION SURABAYA     15
RB05_-_SKBM                  265
RB12__-_CMH                  237
RB16_-_GRSK                  335
RB23_-_KRI

# Cleaning dan Standarisasi Kolom Kategori dan Lokasi Toko

Mapping Data

In [22]:
# Mapping Kategori
MAP_KATEGORI = {
    # === 1. KIDDIE RIDE ===
    '(( KIDDIE RIDE ))': 'KIDDIE RIDE',
    '(( FIXED GAME / KIDDIE RIDE ))': 'KIDDIE RIDE',
    '(( FIXED GAME/KIDDIE RIDE))': 'KIDDIE RIDE',
    '((FIXED GAME/ KIDDIE RIDE ))': 'KIDDIE RIDE',

    # === 2. LARGE FIXED GAME ===
    '(( FIXED GAME ))': 'LARGE FIXED GAME',
    '(( FIXED GAME / FIXED GAME ))': 'LARGE FIXED GAME',
    '(( FIXED GAME))': 'LARGE FIXED GAME',
    '(( LARGE FIXED GAME ))': 'LARGE FIXED GAME',
    '(( LARGE FIXED GAME )) *BATREY CAR MODEL BBC AKI MODEL JADUL': 'LARGE FIXED GAME',
    '((LARGE FIXED GAME ))': 'LARGE FIXED GAME',
    '((LARGE FIXED GAME))': 'LARGE FIXED GAME',

    # === 3. LARGE GAME (Logika Utama ada di fungsi, ini untuk yg kalimat panjang) ===
    '(( LARGE GAME ))': 'LARGE GAME',
    '(( LARGE  GAME ))  ST. MALANG – RBM': 'LARGE GAME',
    # Kalimat Upgrade Panjang = LARGE GAME
    'DI UP GRADE MENJADI DX SINGLE BULAN MEI 2009': 'LARGE GAME',
    'DI UP GRADE MENJADI DX SINGLE BULAN SEPTEMBER 2009': 'LARGE GAME',
    'DI UP-GRADE MENJADI PUMP IT UP FIESTA IX 2011  PER MARET 2011': 'LARGE GAME',
    'DI UP-GRADE MENJADI PUMP IT UP FIESTA IX 2011 PER 2011': 'LARGE GAME',
    'DI UP-GRADE MENJADI PUMP IT UP FIESTA IX 2011 PER MARET 2011': 'LARGE GAME',
    'DI UP-GRADE MENJADI PUMP IT UP FIESTA PER MARET 2011': 'LARGE GAME',
    'SUDAH DI UP GRADE JADI PUMP IT UP ABSOLUT TGL 20 JULI 2009': 'LARGE GAME',
    'SUDAH DI UP GRADE MENJADI DELUXE SD SINGLE PER TGL 18 NOVEMBER 2009': 'LARGE GAME',
    'SUDAH DI UP GRADE MENJADI DELUXE SD SINGLE PER TGL 25 NOVEMBER 2009': 'LARGE GAME',
    'SUDAH DI UP GRADE MENJADI DELUXE SD SINGLE PER TGL 26 NOVEMBER 2009': 'LARGE GAME',
    'SUDAH DI UP GRADE MENJADI DELUXE SD SINGLE PER TGL 28 NOVEMBER 2009': 'LARGE GAME',

    # === 4. PRIZE GAME ===
    '(( PRIZE GAME )': 'PRIZE GAME',
    '(( PRIZE GAME ))': 'PRIZE GAME',
    '(( PRIZE GAME))': 'PRIZE GAME',
    '((PRIZE GAME ))': 'PRIZE GAME',
    '((PRIZE GAME))': 'PRIZE GAME',
    # Kalimat Khusus
    'KET  ANIMAL KAISSER VER. 3 DI UP GRADE MENJADI VER. 6 (06 JUNI 2011)': 'PRIZE GAME',

    # === 5. PUSHER GAME ===
    '(( PUSHER GAME ))': 'PUSHER GAME',
    '(( PUSHER MACHINE ))': 'PUSHER GAME',
    '((PUSHER GAME))': 'PUSHER GAME',
    '((PUSHER MACHINE))': 'PUSHER GAME',

    # === 6. REDEMPTION ===
    '(( REDEMPTION ))': 'REDEMPTION',
    '(( REDEMPTION GAME ))': 'REDEMPTION',
    '(( REDEMTION  ))': 'REDEMPTION', # Typo spasi
    '(( REDEMTION ))': 'REDEMPTION', # Typo
    '((REDEMPTION))': 'REDEMPTION',
    'TITIP JADI HAK MILIK R 34': 'REDEMPTION', # Request Khusus

    # === 7. KARAOKE ===
    '(( KARAOKE )': 'KARAOKE',
    '(( KARAOKE ))': 'KARAOKE',
    '(( KARAOKE - BARANG SAJA/UNIT CPU ))': 'KARAOKE',
    '(( KARAOKE - REAL TIME/KESATUAN))': 'KARAOKE',
    '(( KARAOKE ROOM ))': 'KARAOKE',
    '((KARAOKE ))': 'KARAOKE',
    '((KARAOKE))': 'KARAOKE',

    # === 8. KURSI PIJAT ===
    '((  KURSI PIJAT ))': 'KURSI PIJAT',
    '(( KURSI PIJAT ))': 'KURSI PIJAT',
    '((KURSI PIJAT ))': 'KURSI PIJAT',
    '((KURSI PIJAT))': 'KURSI PIJAT',

    # === 9. KIDDIE LAND (Soft Play & Playground) ===
    '(( KIDDIE LAND ))': 'KIDDIE LAND',
    '(( SOFT PLAY ))': 'KIDDIE LAND',
    '((SOFT PLAY))': 'KIDDIE LAND',
    '(( SOFT PLAYA / KIDDIE LAND ))': 'KIDDIE LAND',
    '(( SOFTPLAY / KIDDIE LAND ))': 'KIDDIE LAND',
    '(( WAHAHA KIDDIE LAND ))': 'KIDDIE LAND',
    '(( WAHANA KIDDI LAND ))': 'KIDDIE LAND',
    '(( WAHANA KIDDIE  LAND ))': 'KIDDIE LAND',
    '(( WAHANA KIDDIE LAND ))': 'KIDDIE LAND',
    '(( WAHANA KIDDIELAND ))': 'KIDDIE LAND',
    '(( WAHANA - MENJADI KIDDIE LAND ))': 'KIDDIE LAND',
    # Variasi Kalimat Upgrade Wahana -> Kiddie Land
    '(( WAHANA - MENJADI SEMI KIDDIE LAND 12 MARET 2017 ))': 'KIDDIE LAND',
    '(( WAHANA - MENJADI SEMI KIDDIE LAND 18 FEBRUARI 2017 )': 'KIDDIE LAND',
    '(( WAHANA - MENJADI SEMI KIDDIE LAND 22 JANUARI 2017 ))': 'KIDDIE LAND',
    '(( WAHANA > MENJADI SEMI KIDDIE LAND 12 MARET 2017 ))': 'KIDDIE LAND',
    '(( WAHANA > MENJADI SEMI KIDDIE LAND 28 DESEMBER 2016 )': 'KIDDIE LAND',

    # === 10. WAHANA ===
    '(( WAHANA  ))': 'WAHANA',
    '(( WAHANA ))': 'WAHANA',
    '((WAHANA ))': 'WAHANA',
    '((WAHANA))': 'WAHANA',
    '(( SOFT PLAY / WAHANA ))': 'WAHANA', # Dominan Wahana
    '(( WAHANA / SOFT PLAY ))': 'WAHANA',

    # === 11. PHOTOBOOTH ===
    '((  PHOTOBOOTH ))': 'PHOTOBOOTH',
    '(( PHOTOBOOTH ))': 'PHOTOBOOTH',
    '(( PHOTOSTUDIO ))': 'PHOTOBOOTH',
    '((PHOTOBOOTH ))': 'PHOTOBOOTH',
    '((PHOTOBOOTH))': 'PHOTOBOOTH',

    # === 12. LAIN-LAIN & UNKNOWN ===
    '(( LAIN - LAIN ))': 'LAIN-LAIN',
    '(( LAIN – LAIN ))': 'LAIN-LAIN',
    '(( LAIN-LAIN ))': 'LAIN-LAIN',
    'UNCATEGORIZED': 'UNCATEGORIZED'
}

In [23]:
# Mapping Lokasi
MAP_LOKASI = {
    # R Jawa
    'R 15 BGR2': 'R015 Dewi Sartika',
    'R 20 CPUTAT': 'R020 Ciputat',
    'R 21 BKSI': 'R021 Bekasi',
    'R 22 PSMGU': 'R022 Pasar Minggu',
    'R 26 - CIMONE': 'R026 Cimone',
    'R26 - CIMONE': 'R026 Cimone',
    'R 29 - TJPR': 'R029 Tanjung Priuk',
    'R 31 - TANGGERANG': 'R031 Tangerang',
    'R 34 DPK': 'R034 Depok',
    'R 35 CLGN': 'R035 Cilegon',
    'R 37 CBTG': 'R037 Cibitung',
    'R 38 KLD': 'R038 Klender 1',
    'R 40 CKRG': 'R040 Cengkareng',
    'R40 CKRG': 'R040 Cengkareng',
    'R 49 CLDG': 'R049 Ciledug',
    'R 67 KARWG': 'R067 Karawang',
    'R 71 BTM': 'R071 BTM',
    'R 75 SRNG': 'R075 Serang',
    'R 100 CRBN': 'R100 Cirebon',
    'R 102 KBL2': 'R102 Kebayoran Lama',
    'R 105 - PRG': 'R105 Parung',
    'R106_-_CBDK': 'R106 Cibadak',
    'R_110_-_PEKALONGAN': 'R110 Pekalongan',
    'R114 - BGR JASMIN': 'R114 Bogor Yasmin',
    'R 121 - TAJUR': 'R121 Tajur',
    'R 124 - CIKUPA': 'R124 Cikupa',
    'R 125 - JTNG': 'R125 Jatinegara',
    'R 135 - SS TANGERANG': 'R135 SS Tangerang',

    # === R (LUAR JAWA & CABANG LAIN) ===
    'R_41_-_PMTS2': 'R041 Siantar',
    'R_45_-_JAMBI': 'R045 Jambi',
    'R_48_-_LMPG': 'R048 Lampung',
    'R_51_-_BLPP': 'R051 Balikpapan',
    'R_52_-_PKPNG': 'R052 Pangkal Pinang',
    'R_56_-_PKB_NEW': 'R056 Pekanbaru',
    'R_57_-_PNTK': 'R057 Pontianak',
    'R_58_-_MDN2': 'R058 Medan 2',
    'R_60_-_TRKN': 'R060 Tarakan',
    'R_61_-_KPG': 'R061 Kupang',
    'R_63_-_BNJMS': 'R063 Banjarmasin',
    'R_66_-_PDG2': 'R066 Padang',
    'R_70_-_MKSR2': 'R070 Makassar 2',
    'R_77_-_BKTG': 'R077 Bukittinggi',
    'R_79_-_MKSR3': 'R079 Makassar 3',
    'R_81_-_BONTG': 'R081 Bontang',
    'R_83_-BYWGI': 'R083 Banyuwangi',
    'R_86__-_KRCI': 'R086 Kerinci',
    'R_89_-_TBTG': 'R089 Tebing Tinggi',
    'R_91_ABPR': 'R091 Abepura',
    'R_94_-_SMRNDA': 'R094 Samarinda',
    'R_96_-__PANAM': 'R096 Panam',
    'R_98_KDRI': 'R098 Kediri',
    'R_99_-_GRUT': 'R099 Garut',
    'R_101_-_LMPG2': 'R101 Rajabasa',
    'R_103_-_SRONG': 'R103 Sorong',
    'R_119_-_MLG_DINOYO': 'R119 Dinoyo',
    'R138_-_TIMIKA': 'R138 Timika', # <--- SUDAH DIKOREKSI

    # RB
    'RB05_-_SKBM': 'RB05 Sukabumi',
    'RB12__-_CMH': 'RB12 Cimahi',
    'RB16_-_GRSK': 'RB16 Gresik',
    'RB_20_-_DPS': 'RB20 Denpasar',
    'RB23_-_KRIAN': 'RB23 Krian',
    'RB27__-_SDJ': 'RB27 Sidoarjo',
    'RB28__-_KDS': 'RB28 Kudus',
    'RB_30_-_PANBIL': 'RB30 Panbil',
    'RB 31 BG JUCTION SURABAYA': 'RB31 Surabaya',
    'RBM_-_MLG': 'RBM Malang',

    # === ZD
    'ZD_-_15_BAGUS_BALI_2': 'ZD15 Bali Batu Bulan',
    'ZD_25_-_WONOSARI_': 'ZD25 Wonosari',
    'ZD37_-_MESRA_SAMARINDA': 'ZD37 Samarinda',
    'ZD_39_-_PAYAKUMBUH2': 'ZD39 Payakumbuh',
    'ZD44_-_PONOROGO': 'ZD44 Ponorogo',
    'ZD46 LOTTE BINTARO': 'ZD46 Lotte Bintaro'
}

Jalankan Fungsi Cleaning

In [24]:
def bersihkan_teks(teks, kamus_mapping):
    if pd.isna(teks): return None
    
    # Bersihkan spasi & Uppercase
    teks_raw = str(teks).strip().upper()
    
    # A. Cek Mapping Langsung (Prioritas Utama)
    if teks_raw in kamus_mapping:
        return kamus_mapping[teks_raw]
    
    # B. LOGIKA DETEKSI OTOMATIS (Untuk Kategori yang variatif)
    if "LARGE GAME" in teks_raw:
        return "LARGE GAME"
    if "KIDDIE RIDE" in teks_raw:
        return "KIDDIE RIDE"
    if "FIXED GAME" in teks_raw:
        return "LARGE FIXED GAME"
        
    # C. Cek Mapping Tanpa Kurung (Backup)
    teks_tanpa_kurung = teks_raw.replace("((", "").replace("))", "").strip()
    if teks_tanpa_kurung in kamus_mapping:
        return kamus_mapping[teks_tanpa_kurung]

    # Default: Kembalikan teks asli (yang sudah di-upper)
    return teks_raw

Terapkan ke Kategori dan Lokasi

In [25]:
# --- 2. TERAPKAN KE MASTER ASET ---
# Bersihkan Kategori
df_master['kategori'] = df_master['kategori'].apply(lambda x: bersihkan_teks(x, MAP_KATEGORI))
# Bersihkan Lokasi Toko
df_master['lokasi_toko'] = df_master['lokasi_toko'].apply(lambda x: bersihkan_teks(x, MAP_LOKASI))

# --- 3. TERAPKAN KE RIWAYAT LOG ---
# Bersihkan Kategori
df_history['kategori'] = df_history['kategori'].apply(lambda x: bersihkan_teks(x, MAP_KATEGORI))
# Bersihkan Lokasi Asal (Ingat, nama kolomnya beda di sini)
df_history['lokasi_asal'] = df_history['lokasi_asal'].apply(lambda x: bersihkan_teks(x, MAP_LOKASI))

Handle Data Kosong

In [26]:
# Mengisi sel kosong dengan NULL agar MySQL tidak bingung
df_master = df_master.replace({np.nan: None})
df_history = df_history.replace({np.nan: None})

Simpan ke File xlsx clean

In [27]:
file_master_clean = '1_Master_Aset_Cleaned.xlsx'
file_history_clean = '2_Riwayat_Log_Cleaned.xlsx'

df_master.to_excel(file_master_clean, index=False)
df_history.to_excel(file_history_clean, index=False)

print("\n" + "="*40)
print(f"1. {file_master_clean} (Jumlah: {len(df_master)} baris)")
print(f"2. {file_history_clean} (Jumlah: {len(df_history)} baris)")
print("="*40)


1. 1_Master_Aset_Cleaned.xlsx (Jumlah: 6100 baris)
2. 2_Riwayat_Log_Cleaned.xlsx (Jumlah: 10821 baris)


Preview Hasil Lokasi & Kategori

In [28]:
print("\n--- Cek Sampel Hasil Cleaning (Master Aset) ---")
print(df_master[['lokasi_toko', 'kategori']].head(10))


--- Cek Sampel Hasil Cleaning (Master Aset) ---
         lokasi_toko    kategori
0  R015 Dewi Sartika  LARGE GAME
1  R015 Dewi Sartika  LARGE GAME
2  R015 Dewi Sartika  LARGE GAME
3  R015 Dewi Sartika  LARGE GAME
4  R015 Dewi Sartika  LARGE GAME
5  R015 Dewi Sartika  REDEMPTION
6  R015 Dewi Sartika  REDEMPTION
7  R015 Dewi Sartika  REDEMPTION
8  R015 Dewi Sartika  REDEMPTION
9  R015 Dewi Sartika  REDEMPTION
